In [1]:
import os
os.chdir("../../")


import torch
from pair_classification.bert_finetuning.finetune import set_configs, timer
from pair_classification.bert_finetuning.triplets.hugginface_pair_clf import BertForSequenceTriplets
from pair_classification.bert_finetuning.triplets.finetune import parse_data_to_bert_format, train, setup_bert_model

%load_ext autoreload
%autoreload 2

torch.cuda.is_available()


True

In [2]:
config = set_configs('models/bert_finetuning/config_triplets.yml')
device = torch.device(f"cuda:{config['torch_device']}")

## Tokenization for triplets. Data already splitted.

In [3]:
with timer('Read data and convert to BERT format'):
    train_loader = parse_data_to_bert_format(
        path_to_data=config['path_to_data'],
        train_file_name=config['train_file_name'],
        anchor_col_name=config['anchor_col_name'],
        pos_col_name=config['pos_col_name'],
        neg_col_name=config['neg_col_name'],
        path_to_pretrained_model=config['path_to_pretrained_model'],
        max_seq_length=config['max_seq_length'],
        batch_size=config['batch_size'],
        toy=config['toy']
    )


loaded 660042 train records


  0%|          | 0/660042 [00:00<?, ?it/s]

There are 0 lines longer than 148


  0%|          | 929/660042 [00:00<02:25, 4530.11it/s]

There are 0 lines longer than 148


100%|██████████| 660042/660042 [02:03<00:00, 5341.94it/s]


There are 2 lines longer than 148
[Read data and convert to BERT format] done in 381 s


## Prepare bert model. Load pretrained weights.

In [4]:
with timer('Setting up BERT model'):
    model, optimizer = setup_bert_model(
        path_to_pretrained_model=config['path_to_pretrained_model'],
        epochs=config['epochs'],
        lrate=config['lrate'],
        lrate_clf=config['lrate_clf'],
        batch_size=config['batch_size'],
        accum_steps=config['accum_steps'],
        lin_dim=config['lin_dim'],
        lin_dropout_prob=config['lin_dropout_prob'],
        warmup=config['warmup'],
        apex_mixed_precision=config['apex_mixed_precision'],
        seed=config['seed'], device=device, train_loader=train_loader,
        clf_class=BertForSequenceTriplets
    )


Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /home/ubuntu/projects/kaggle/quora_question_pairs_kaggle/models/bert_finetuning/uncased_L-12_H-768_A-12/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_0

Loading TF weight bert/encoder/layer_4/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_4/output/LayerNorm/gamma with shape [768]
Loading TF weight bert/encoder/layer_4/output/dense/bias with shape [768]
Loading TF weight bert/encoder/layer_4/output/dense/kernel with shape [3072, 768]
Loading TF weight bert/encoder/layer_5/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_5/attention/output/LayerNorm/gamma with shape [768]
Loading TF weight bert/encoder/layer_5/attention/output/dense/bias with shape [768]
Loading TF weight bert/encoder/layer_5/attention/output/dense/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_5/attention/self/key/bias with shape [768]
Loading TF weight bert/encoder/layer_5/attention/self/key/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_5/attention/self/query/bias with shape [768]
Loading TF weight bert/encoder/layer_5/attention/self/query/kernel with shape [768, 76

[Setting up BERT model] done in 12 s


## Metric learning. Training 2 epochs.

In [ ]:
with timer('Training'):
    model = train(
        model=model,
        optimizer=optimizer,
        epochs=1,
        accum_steps=config['accum_steps'],
        apex_mixed_precision=config['apex_mixed_precision'],
        output_model_file_name=config['output_model_file_name'],
        device=device,
        train_loader=train_loader,
        bert_freezed=False
    )


  0%|          | 0/20627 [00:00<?, ?it/s]

Iter 207/20627. Postfix = loss=5.79
Iter 414/20627. Postfix = loss=4.83
Iter 621/20627. Postfix = loss=4.14
Iter 828/20627. Postfix = loss=3.6
Iter 1035/20627. Postfix = loss=2.8
Iter 1242/20627. Postfix = loss=2.33
Iter 1449/20627. Postfix = loss=1.38
Iter 1656/20627. Postfix = loss=1.15
Iter 1863/20627. Postfix = loss=0.888
Iter 2070/20627. Postfix = loss=0.768
Iter 2277/20627. Postfix = loss=0.73
Iter 2484/20627. Postfix = loss=0.675
Iter 2691/20627. Postfix = loss=0.641
Iter 2898/20627. Postfix = loss=0.641
Iter 3105/20627. Postfix = loss=0.548
Iter 3312/20627. Postfix = loss=0.533
Iter 3519/20627. Postfix = loss=0.515
Iter 3726/20627. Postfix = loss=0.548
Iter 3933/20627. Postfix = loss=0.5
Iter 4140/20627. Postfix = loss=0.473
Iter 4347/20627. Postfix = loss=0.484
Iter 4554/20627. Postfix = loss=0.467
Iter 4761/20627. Postfix = loss=0.479
Iter 4968/20627. Postfix = loss=0.441
Iter 5175/20627. Postfix = loss=0.47
Iter 5382/20627. Postfix = loss=0.443
Iter 5589/20627. Postfix = los

In [ ]:
torch.save(model.state_dict(), config['path_to_output_model'] + 'bert_triplets_unfreezed.bin')

In [ ]:
with timer('Training'):
    model = train(
        model=model,
        optimizer=optimizer,
        epochs=1,
        accum_steps=config['accum_steps'],
        apex_mixed_precision=config['apex_mixed_precision'],
        output_model_file_name=config['output_model_file_name'],
        device=device,
        train_loader=train_loader,
        bert_freezed=False
    )


In [ ]:
torch.save(model.state_dict(), config['path_to_output_model'] + 'bert_triplets_unfreezed_2.bin')

### Training logs are lost. Loss after unfreezed 2nd epoch ~ 0.07